In [1]:
import src.optimization_package.optimizer
from src.optimization_package.optimizer import OptimizerPGPE
import config.CONSTANTS_HMM
from config.CONSTANTS_HMM import N_camp
import importlib
import numpy as np
from joblib import delayed, Parallel

In [2]:
importlib.reload(src.optimization_package.optimizer)
from src.optimization_package.optimizer import OptimizerPGPE
importlib.reload(config.CONSTANTS_HMM)
from config.CONSTANTS_HMM import N_camp

In [3]:
uniform_initialization = {
    'tot_impressions': 5000,
    'num_campaigns': N_camp
}

policy = OptimizerPGPE(num_sample=64, hmm_trials=20, N_users_to_simulate=5000, uniform_initialization=uniform_initialization)

In [4]:
budgets_list = []
convs_list = np.array([])
learning_rate = 1

while True:
    budgets = policy.get_theta()
    returns = Parallel(n_jobs=4)(delayed(policy.act)(budget) for budget in budgets)

    print(returns)

    delta_params = np.array(policy.eval_gradient(budgets, returns, use_baseline=True))

    print('Budget distribution: ', policy.get_budget_distribution(),         'Return: ', policy.act(policy.get_budget_distribution()),
          'Variance: ', policy.variance)

    params = policy.get_params()
    policy.set_params(params + learning_rate*delta_params)

    learning_rate = max(0.95*learning_rate, 1e-3)
    #convs_list *= 0.99

    if max(policy.variance) < 1e-4:
        break

2022-06-12 18:44:24.856966: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-12 18:44:24.868412: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-12 18:44:24.965000: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

[0.05157, 0.09997, 0.43666, 0.09437, 0.09661, 0.09546, 0.04683, 0.1241, 0.66821, 0.084, 0.80392, 0.10664, 0.79013, 0.58645, 0.57183, 0.12935, 0.12615, 0.70333, 0.16523, 0.03369, 0.08485, 0.49312, 0.38817, 0.25942, 0.25181, 0.20929, 0.43561, 0.14213, 0.80898, 0.66013, 0.097, 0.34444, 0.47776, 0.57498, 0.34928, 0.0407, 0.79238, 0.49308, 0.79423, 0.43652, 0.8115, 0.1042, 0.37769, 0.10087, 0.09427, 0.16967, 0.80425, 0.34327, 0.28724, 0.34159, 0.12507, 0.14965, 0.05055, 0.06033, 0.72675, 0.26264, 0.06025, 0.13673, 0.42686, 0.09675, 0.03016, 0.48118, 0.09183, 0.49604]


2022-06-12 18:46:50.914638: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Budget distribution:  [0.25 0.25 0.25 0.25] Return:  0.36467 Variance:  [2.4 2.4 2.4 2.4]
[0.35841, 0.35023, 0.38472, 0.79093, 0.23692, 0.20211, 0.15735, 0.6769, 0.28125, 0.05563, 0.52767, 0.21547, 0.293, 0.76364, 0.20908, 0.50438, 0.6358, 0.43549, 0.78851, 0.22745, 0.18164, 0.2882, 0.0862, 0.6221, 0.56341, 0.11678, 0.21167, 0.56536, 0.15933, 0.71274, 0.64033, 0.16059, 0.23842, 0.21566, 0.18051, 0.39348, 0.24707, 0.55523, 0.19063, 0.17429, 0.77564, 0.67966, 0.79605, 0.77455, 0.1077, 0.14204, 0.31443, 0.16, 0.68596, 0.26846, 0.76841, 0.63489, 0.73333, 0.18567, 0.26262, 0.66418, 0.39006, 0.04921, 0.18846, 0.34251, 0.60928, 0.29454, 0.18311, 0.59068]
Budget distribution:  [0.22924204 0.2597255  0.28136037 0.22967208] Return:  0.40435 Variance:  [2.06429304 1.66450769 1.44165431 2.05778805]
[0.55705, 0.63262, 0.66626, 0.25365, 0.52911, 0.65763, 0.27141, 0.36945, 0.56627, 0.585, 0.43025, 0.55154, 0.34394, 0.3475, 0.25451, 0.45275, 0.61887, 0.49365, 0.1509, 0.63973, 0.54134, 0.39046, 0.66371

Exception in thread ExecutorManagerThread:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/Thom/Library/Caches/JetBrains/PyCharm2021.3/demo/PyCharmLearningProject/venv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 575, in run
    self.flag_executor_shutting_down()
  File "/Users/Thom/Library/Caches/JetBrains/PyCharm2021.3/demo/PyCharmLearningProject/venv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 770, in flag_executor_shutting_down
    self.kill_workers(reason="executor shutting down")
  File "/Users/Thom/Library/Caches/JetBrains/PyCharm2021.3/demo/PyCharmLearningProject/venv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 781, in kill_workers
    recursive_terminate(p)
  File "/Users/Thom/Library/Caches/JetBrains/PyCharm2021.3/demo/PyCharmLearningProject/venv/l

In [ ]:
np.mean([0.45989, 0.466, 0.48327, 0.46501, 0.4779, 0.46358, 0.47808, 0.48013, 0.48484, 0.45388, 0.4657, 0.47299, 0.45442, 0.4719, 0.47902, 0.4875, 0.47216, 0.47563, 0.48212, 0.47095, 0.46474, 0.45953, 0.48298, 0.48572, 0.46531, 0.49117, 0.46822, 0.47105, 0.48765, 0.46717, 0.4613, 0.47137, 0.4725, 0.48674, 0.48742, 0.48113, 0.46859, 0.49692, 0.46038, 0.49049, 0.4961, 0.46852, 0.48149, 0.47633, 0.48159, 0.46804, 0.48147, 0.47345, 0.47052, 0.48261, 0.45707, 0.47309, 0.47618, 0.4826, 0.47049, 0.46822, 0.47146, 0.46319, 0.47802, 0.47406, 0.47052, 0.48033, 0.46026, 0.46729])